Carichiamo i dati e iniziamo a fare qualche carotaggio per verificare quali informazioni sono presenti

In [20]:
import pandas as pd
import glob
from datetime import datetime


In [21]:
dateparse = lambda x: datetime.strptime(x, '%d/%m/%Y')

In [22]:
datafiles = glob.glob('Rapporti_di_lavoro_*.csv')

In [24]:
files = []
for file in datafiles:
    df_tmp = pd.read_csv(file, parse_dates=['DATA'], date_parser=dateparse)
    df_tmp['TIPOLOGIA'] = file[19:-4]
    files.append(df_tmp)

frame = pd.concat(files, axis=0, ignore_index=True)

In [25]:
# TODO: la lettura della data col date parser è da rivedere, ci sono probabilmente delle date che non sono scritte nel formato del paser
frame

,DATA,GENERE,ETA,SETTOREECONOMICODETTAGLIO,TITOLOSTUDIO,CONTRATTO,MODALITALAVORO,PROVINCIAIMPRESA,ITALIANO,TIPOLOGIA
0,2019-04-02 00:00:00,M,20,Lavori di meccanica generale,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,APPRENDISTATO PROFESSIONALIZZANTE O CONTRATTO ...,TEMPO PIENO,BERGAMO,ITALIA,cessati
1,2018-06-08 00:00:00,F,28,Fabbricazione di altre apparecchiature elettri...,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,APPRENDISTATO PROFESSIONALIZZANTE O CONTRATTO ...,TEMPO PIENO,BERGAMO,ITALIA,cessati
2,2016-02-02 00:00:00,F,20,Fabbricazione di altre apparecchiature elettri...,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,APPRENDISTATO PROFESSIONALIZZANTE O CONTRATTO ...,TEMPO PIENO,BERGAMO,ITALIA,cessati
3,2015-01-12 00:00:00,M,59,"Gestioni di funicolari, ski-lift e seggiovie s...",LICENZA MEDIA,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,BERGAMO,ITALIA,cessati
4,2017-08-09 00:00:00,M,61,"Gestioni di funicolari, ski-lift e seggiovie s...",LICENZA MEDIA,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,BERGAMO,ITALIA,cessati
...,...,...,...,...,...,...,...,...,...,...
18710142,2017-08-09 00:00:00,M,22,Commercio al dettaglio di carburante per autot...,NESSUN TITOLO DI STUDIO,LAVORO A TEMPO DETERMINATO,TEMPO PARZIALE ORIZZONTALE,BRESCIA,PAKISTAN,trasformati
18710143,2019-10-01 00:00:00,F,42,Produzione di software non connesso all'edizione,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,TIROCINIO,TEMPO PIENO,MONZA E BRIANZA,ITALIA,trasformati
18710144,2012-06-02 00:00:00,M,40,Lavori di meccanica generale,LICENZA MEDIA,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,MONZA E BRIANZA,ITALIA,trasformati
18710145,2021-05-02 00:00:00,M,21,Fabbricazione di altre pompe e compressori,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,MONZA E BRIANZA,PAKISTAN,trasformati


In [33]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18710147 entries, 0 to 18710146
Data columns (total 10 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   DATA                       object
 1   GENERE                     object
 2   ETA                        int64 
 3   SETTOREECONOMICODETTAGLIO  object
 4   TITOLOSTUDIO               object
 5   CONTRATTO                  object
 6   MODALITALAVORO             object
 7   PROVINCIAIMPRESA           object
 8   ITALIANO                   object
 9   TIPOLOGIA                  object
dtypes: int64(1), object(9)
memory usage: 1.4+ GB


In [39]:
frame.sort_values(by='DATA', inplace=True)

KeyboardInterrupt: 

In [36]:
frame['ANNO'] = pd.DatetimeIndex(frame['DATA']).year
frame['MESE'] = pd.DatetimeIndex(frame['DATA']).month

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 2600-11-12 00:00:00

In [34]:
frame.groupby(['SETTOREECONOMICODETTAGLIO']).size()


SETTOREECONOMICODETTAGLIO
Acquacoltura in acqua di mare, salmastra o lagunare e servizi connessi                                1642
Acquacoltura in acque dolci e servizi connessi                                                         868
Affari esteri                                                                                         1277
Affittacamere per brevi soggiorni, case ed appartamenti per vacanze, bed and breakfast, residence    14418
Affitto di aziende                                                                                     772
                                                                                                     ...  
Trattamento igienico del latte                                                                        2268
Trivellazioni e perforazioni                                                                          4671
Università popolare                                                                                     94
Utilizzo di

In [14]:
ateco = pd.read_excel('CodiceAteco.xlsx')
ateco

In [16]:
frame_ateco = frame.merge(ateco, left_on='SETTOREECONOMICODETTAGLIO', right_on='Descrizione', how='left', indicator=True)

In [19]:
frame_ateco.groupby(['TIPOLOGIA']).size()

TIPOLOGIA
attivati       9579794
cessati        3741394
prorogati      3413617
trasformati    1975342
dtype: int64

In [39]:
pd.crosstab(frame_ateco.Lettera, frame_ateco.GENERE)

GENERE,F,M
Lettera,,
A,5220,20166
B,88,516
C,60710,148446
D,615,885
E,1186,7725
F,5059,88299
G,118009,81835
H,38280,124931
I,93272,109798


In [41]:
frame_ateco.DATA.max()

'12/12/2020'

In [42]:
frame_ateco.DATA.min()

'01/01/2010'

In [45]:
pd.crosstab(frame_ateco.TITOLOSTUDIO, frame_ateco.GENERE, margins=True)

GENERE,F,M,All
TITOLOSTUDIO,,,
DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CHE PERMETTE L'ACCESSO ALL'UNIVERSITA,302834,213558,516392
DIPLOMA DI SPECIALIZZAZIONE,5501,4111,9612
DIPLOMA TERZIARIO EXTRA-UNIVERSITARIO,655,401,1056
DIPLOMA UNIVERSITARIO,9142,5174,14316
LAUREA - Vecchio o nuovo ordinamento,149645,69040,218685
LICENZA ELEMENTARE,3729,14309,18038
LICENZA MEDIA,194024,340259,534283
MASTER UNIVERSITARIO DI PRIMO LIVELLO,1246,653,1899
NESSUN TITOLO DI STUDIO,161994,322216,484210
